In [0]:
%sql
CREATE DATABASE IF NOT EXISTS iot;
USE iot;
DROP TABLE IF EXISTS IotDeviceData;
CREATE OR REPLACE TABLE IotDeviceData (device STRING, status STRING );
SHOW TABLES;

database,tableName,isTemporary
iot,iotdevicedata,false


In [0]:
%sql
DESCRIBE iotdevicedata;

col_name,data_type,comment
device,string,null
status,string,null


In [0]:
%sql
DESCRIBE EXTENDED IotDeviceData;

col_name,data_type,comment
device,string,null
status,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,iot,
Table,IotDeviceData,
Created Time,Sun Jul 07 10:05:07 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.5.0,


In [0]:
%sql
DROP TABLE IF EXISTS IotDeviceData;

In [0]:
delta_stream_table_path = "dbfs:/dbfs/delta/iotdevicedata"

In [0]:
spark.sql("CREATE OR REPLACE TABLE IotDeviceData (device STRING, status STRING ) USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

DataFrame[]

In [0]:
%sql
DESCRIBE EXTENDED IotDeviceData;

col_name,data_type,comment
device,string,null
status,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,iot,
Table,IotDeviceData,
Created Time,Sun Jul 07 10:05:11 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.5.0,


In [0]:
%sql
SELECT * FROM IotDeviceData;

device,status


In [0]:
!rm -r /dbfs/device_stream/

rm: cannot remove '/dbfs/device_stream/': No such file or directory


In [0]:
dbutils.fs.mkdirs('dbfs:/dbfs/device_stream')
dbutils.fs.mkdirs('dbfs:/dbfs/device_stream')

dbutils.fs.mkdirs('dbfs:/dbfs/delta/iotdevicedata')
dbutils.fs.mkdirs('dbfs:/dbfs/delta/iotdevicedata')

dbutils.fs.mkdirs('dbfs:/dbfs/delta/checkpoint')
dbutils.fs.mkdirs('dbfs:/dbfs/delta/checkpoint')

True

In [0]:
%sh
# rm -r /dbfs/device_stream
# mkdir /dbfs/device_stream

# rm -r /dbfs/delta/iotdevicedata
# mkdir /dbfs/delta/iotdevicedata

# rm -r /dbfs/delta/checkpoint
# mkdir /dbfs/delta/checkpoint

wget -O /dbfs/device_stream/devices1.json https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json

/dbfs/device_stream/devices1.json: No such file or directory


In [0]:
!wget https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json

--2024-07-07 10:06:45--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296 [text/plain]
Saving to: ‘devices1.json’

devices1.json       100%[===================>]     296  --.-KB/s    in 0s      

2024-07-07 10:06:45 (7.06 MB/s) - ‘devices1.json’ saved [296/296]



In [0]:
dbutils.fs.mv("file:/databricks/driver/devices1.json", "dbfs:/dbfs/device_stream/devices1.json")

True

In [0]:
dbutils.fs.ls("dbfs:/dbfs/device_stream")

[FileInfo(path='dbfs:/dbfs/device_stream/devices1.json', name='devices1.json', size=296, modificationTime=1720346827000)]

In [0]:
!ls /dbfs/device_stream/

ls: cannot access '/dbfs/device_stream/': No such file or directory


In [0]:
!cat /dbfs/device_stream/devices1.json

cat: 'dbfs:/dbfs/device_stream/devices1.json': No such file or directory


In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import *


# inputPath = "dbfs:/dbfs/delta/device_stream"
inputPath = "dbfs:/dbfs/device_stream/"
jsonSchema = StructType(
    [
        StructField("device", StringType(), False),
        StructField("status", StringType(), False)
    ]
)
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)
# iotstream = spark.readStream.format("json").schema(jsonSchema).option("maxFilesPerTrigger", 1).load(inputPath)
print("Source stream created")

Source stream created


In [0]:
delta_stream_table_path = "dbfs:/dbfs/delta/iotdevicedata"
checkpointpath = "dbfs:/dbfs/delta/checkpoint"
deltastream = iotstream.writeStream.format("delta").outputMode("append").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
# deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

Streaming to delta sink...


In [0]:
dbutils.fs.ls('dbfs:/dbfs/delta/iotdevicedata')

[FileInfo(path='dbfs:/dbfs/delta/iotdevicedata/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dbfs/delta/iotdevicedata/part-00000-49972b7a-85d7-45cd-a8c5-6dfddb9fc79d-c000.snappy.parquet', name='part-00000-49972b7a-85d7-45cd-a8c5-6dfddb9fc79d-c000.snappy.parquet', size=795, modificationTime=1720346949000)]

In [0]:
dbutils.fs.ls(delta_stream_table_path)

[FileInfo(path='dbfs:/dbfs/delta/iotdevicedata/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dbfs/delta/iotdevicedata/part-00000-49972b7a-85d7-45cd-a8c5-6dfddb9fc79d-c000.snappy.parquet', name='part-00000-49972b7a-85d7-45cd-a8c5-6dfddb9fc79d-c000.snappy.parquet', size=795, modificationTime=1720346949000)]

In [0]:
dbutils.fs.ls('dbfs:/dbfs/delta/checkpoint')

[FileInfo(path='dbfs:/dbfs/delta/checkpoint/commits/', name='commits/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dbfs/delta/checkpoint/metadata', name='metadata', size=45, modificationTime=1720340688000),
 FileInfo(path='dbfs:/dbfs/delta/checkpoint/offsets/', name='offsets/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dbfs/delta/checkpoint/sources/', name='sources/', size=0, modificationTime=0)]

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
iot,iotdevicedata,false


In [0]:
%sql
SELECT * FROM iot.iotdevicedata;

device,status
Dev1,ok
Dev1,ok
Dev1,ok
Dev2,error
Dev1,ok
Dev1,error
Dev2,ok
Dev2,error
Dev1,ok


In [0]:
 !wget https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices2.json

--2024-07-07 10:10:01--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229 [text/plain]
Saving to: ‘devices2.json’

devices2.json       100%[===================>]     229  --.-KB/s    in 0s      

2024-07-07 10:10:01 (4.40 MB/s) - ‘devices2.json’ saved [229/229]



In [0]:
dbutils.fs.mv("file:/databricks/driver/devices2.json", "dbfs:/dbfs/device_stream/devices2.json")

True

In [0]:
dbutils.fs.ls("dbfs:/dbfs/device_stream/devices2.json")

[FileInfo(path='dbfs:/dbfs/device_stream/devices2.json', name='devices2.json', size=229, modificationTime=1720347036000)]

In [0]:
%sql
SELECT * FROM IotDeviceData;

device,status
Dev1,ok
Dev1,ok
Dev1,ok
Dev2,error
Dev1,ok
Dev1,error
Dev2,ok
Dev2,error
Dev1,ok
Dev1,ok


In [0]:
# deltastream.stop()